Run [circular-tank-detector](https://github.com/PlatformStories/circular-tank-detector) over various locations around the world. Details on circular-tank-detector can be found in [this blog post](http://gbdxstories.digitalglobe.com/circular-tanks/).

In [ ]:
# Set credentials

import os
os.environ['GBDX_USERNAME'] = ''
os.environ['GBDX_PASSWORD'] = ''
os.environ['GBDX_CLIENT_ID'] = '' 
os.environ['GBDX_CLIENT_SECRET'] = ''

In [2]:
import gbdxtools
gbdx = gbdxtools.Interface()

First run on a set of small strips. The recommended size is less than 4GB.

In [48]:
catids = ['1030010072C98200',      # Cushing, Oklahoma (WV-02)
          '103001007342CF00',      # Barcelona, Spain (WV-02)
          '10500100088DD800',      # Singapore (GE-01)
          '1030010061A65300',      # Shanghai, China (WV-02)
          '104001002A163800',      # Rotterdam, Netherland (WV-03)  
          '1050050017DD4D00']      # Piraeus, Greece (GE-01)
          
from os.path import join
          
wfids = {}          
          
for catid in catids:          
          
    # order
    order = gbdx.Task('Auto_Ordering', cat_id=catid)
    order.impersonation_allowed = True

    # acomped pansharpened with baselayer matching in utm
    # baselayer matching is optional but results in better colors
    aop = gbdx.Task('AOP_Strip_Processor',
                     data=order.outputs.s3_location.value,
                     enable_dra=False,                      # disable automatic dra
                     ortho_epsg='UTM')                         
    blm = gbdx.Task('baselayermatch',
                    data=aop.outputs.data.value,
                    cloud_id=catid)                        # take clouds into account when dra'ing

    ctd = gbdx.Task('circular-tank-detector',
                    ps_image=blm.outputs.data.value)

    wf = gbdx.Workflow([order, aop, blm, ctd])
    wf.savedata(blm.outputs.data, join('platform-stories/trial-runs', catid))
    wf.savedata(ctd.outputs.detections, join('platform-stories/trial-runs', catid, 'detections'))

    wfids[catid] = wf.execute()

In [57]:
for catid in wfids:
    wf = gbdx.Workflow([])
    wf.id = wfids[catid]
    print wf.id, wf.status

4716506777012140019 {u'state': u'complete', u'event': u'succeeded'}
4716506747749273860 {u'state': u'complete', u'event': u'succeeded'}
4716506764611475361 {u'state': u'complete', u'event': u'succeeded'}
4716506770266181685 {u'state': u'complete', u'event': u'succeeded'}
4716506753489487367 {u'state': u'complete', u'event': u'succeeded'}
4716506758829433586 {u'state': u'complete', u'event': u'succeeded'}


Calculate waiting time and execution time per km2.

In [58]:
import datetime
import numpy as np
import pyproj
from shapely.wkt import loads
from shapely.ops import transform
from functools import partial

# get datetime object from string timestamp
def get_time(timestamp):
    return datetime.datetime.strptime(timestamp[:-13], "%Y-%m-%dT%H:%M:%S")

# get area in km2 from wkt string (we need this for the execution time per km2 calculation)
def area_km2(wkt):
    shape = loads(wkt)
    proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
                   pyproj.Proj('+proj=aea', lat1=shape.bounds[1], lat2=shape.bounds[3]))
    return transform(proj, shape).area/float(10**6)

submission_times, start_times, end_times, failures, wait_times, exec_times = {}, {}, {}, {} , {}, {}

submission_times = {}
start_times = {}
end_times = {} 
failures = {}
wait_times = []
exec_times = []
    
for catid in wfids:
    
    # compute the area
    wkt =  gbdx.catalog.get_strip_footprint_wkt(catid)
    area = area_km2(wkt)
    
    print catid, area
    
    wf = gbdx.Workflow([])
    wf.id = wfids[catid]

    for event in wf.events:

        if 'circular-tank-detector' in event['task']:

            task_id = event['task_id']            
            failures[task_id] = False
            
            if event['event'] == 'submitted':
                submission_times[task_id] = get_time(event['timestamp'])
            elif event['event'] == 'started':
                start_times[task_id] = get_time(event['timestamp'])
            elif event['event'] == 'succeeded':
                end_times[task_id] = get_time(event['timestamp'])
            elif event['event'] == 'failed':
                failures[task_id] = True
    
for task_id in submission_times:

    wait_times.append((start_times[task_id] - submission_times[task_id]).total_seconds()/60)
    if not failures[task_id]:
        exec_times.append((end_times[task_id] - start_times[task_id]).total_seconds()/(60*area))
    
print 'Mean waiting time: {} min'.format(np.mean(wait_times))
print 'Mean execution time: {} min/km2'.format(np.mean(exec_times))

1050050017DD4D00 453.535140827
1030010072C98200 278.978191499
1030010061A65300 325.093946411
104001002A163800 225.424828604
103001007342CF00 449.69804398
10500100088DD800 258.337497285
Mean waiting time: 36.0722222222 min
Mean execution time: 0.205727866767 min/km2


Now run on a set of bigger strips (> 1000 km2). Each strip in broken up in several parts.

In [121]:
catids = ['10400100324CA000',      # Baytown, Texas (WV03)
          '1050010006754D00',      # London, UK (GE-01)
          '10300100633B4900']      # Osaka, Japan (WV-02)   
          
from os.path import join
          
wfids = {}      

tiles = 6
          
for catid in catids: 
              
    # order
    order = gbdx.Task('Auto_Ordering', cat_id=catid)
    order.impersonation_allowed = True

    # acomped pansharpened with baselayer matching in utm
    # baselayer matching is optional but results in better colors
    aop = gbdx.Task('AOP_Strip_Processor',
                     data=order.outputs.s3_location.value,
                     enable_dra=False,                      # disable automatic dra
                     ortho_epsg='UTM')                         
    blm = gbdx.Task('baselayermatch',
                    data=aop.outputs.data.value,
                    cloud_id=catid)                        # take clouds into account when dra'ing

    # crop
    crop = gbdx.Task('CropGeotiff', 
                     data=blm.outputs.data.value, 
                     num_partitions=str(tiles))                
          
    # each of the parts goes to a separate task
    ctd = [0]*tiles   
    ctd[0] = gbdx.Task('circular-tank-detector', ps_image=crop.outputs.data_0.value)
    ctd[1] = gbdx.Task('circular-tank-detector', ps_image=crop.outputs.data_1.value)
    ctd[2] = gbdx.Task('circular-tank-detector', ps_image=crop.outputs.data_2.value)
    ctd[3] = gbdx.Task('circular-tank-detector', ps_image=crop.outputs.data_3.value)
    ctd[4] = gbdx.Task('circular-tank-detector', ps_image=crop.outputs.data_4.value)
    ctd[5] = gbdx.Task('circular-tank-detector', ps_image=crop.outputs.data_5.value)
    
    wf = gbdx.Workflow([order, aop, blm, crop] + ctd)
    
    wf.savedata(blm.outputs.data, join('platform-stories/trial-runs', catid))
    for i in range(tiles):
        wf.savedata(ctd[i].outputs.detections, join('platform-stories/trial-runs', catid , 'detections', str(i)))

    wfids[catid] = wf.execute()